# DX-STREAM Tutorial 1 — Introduction
_Last updated: 2025-10-19 22:39_

### Goals
- Understand what DX-STREAM is and where it fits in the DEEPX SDK
- Learn the core elements (DxPreprocess → DxInfer → DxPostprocess → DxTracker → DxOsd)
- See the model workflow from ONNX to DXNN to runtime

### Prerequisites
- Ubuntu 18.04/20.04/22.04/24.04 (x86_64)
- DX-RT **3.0.0+** installed (driver + runtime)
- GStreamer 1.16+ recommended

> Tip: If you plan to actually run pipelines from the notebook, ensure X11/Wayland forwarding is configured if you're remote.

## DEEPX SDK Architecture (Quick Overview)
- **DX-COM**: compiles ONNX + JSON → **.dxnn** (INT8/optimized) 
- **DX-RT**: runtime that executes **.dxnn** on the NPU (C/C++/Python APIs)
- **DX-STREAM**: GStreamer plugin set for end-to-end vision pipelines
- **ModelZoo / DX-APP**: ready models & examples

## DX-STREAM Pipeline at a Glance
```text
Source → (decodebin) → DxPreprocess → DxInfer → DxPostprocess → DxTracker → DxOsd → Sink
```
- **DxPreprocess**: resize/ROI/color space, optional custom library
- **DxInfer**: runs .dxnn model, links to preprocess via `preprocess-id`
- **DxPostprocess**: decode tensors → metadata, optional custom library
- **DxTracker**: assign stable IDs (e.g., OC_SORT)
- **DxOsd**: draw boxes/labels/poses/segmentation onto frames

## Model Workflow
1. **Compile**: ONNX → `.dxnn` using DX-COM
2. **Execute**: load `.dxnn` via DX-RT inside **DxInfer**
3. **Integrate**: wire preprocess/postprocess/tracking/OSD in a GStreamer pipeline

## Verify Plugin Availability
Run the following to ensure the plugin is registered:


In [ ]:
!gst-inspect-1.0 dxstream || echo 'dxstream plugin not found'

## Best Practices
- Keep element **IDs** (`preprocess-id`, `inference-id`) consistent across Pre/Infer/Post.
- Add **`queue`** between heavy elements to decouple latencies.
- For remote sessions, prefer **`autovideosink`** or use **`sync=false`** to reduce stutter.
- If Rockchip RGA is available, DX-STREAM can offload color/resize/OSD to HW.
- Use JSON configs to keep pipelines reproducible and clean.